In [17]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt # Collect coords into list
import elevation as dem
import os

In [15]:
roi = (51.8903, 10.41933) # Rammelsberg Coordinates (Lat, Lng)
offset = 0.01
bbx = (roi[1]-offset, roi[0]-offset, roi[1]+offset, roi[0]+offset)
fname = f"{os.getcwd()}/ROI-DEM.tif" 
dem.clip(bounds=(bbx[0], bbx[1], bbx[2], bbx[3]), output=fname, product='SRTM1');
dem_data = rio.open(fname)
img = np.array(Image.open(fname))
print(type(img))
print(img.dtype)
print(img.shape)
print(img)

make: Entering directory '/home/ahmad/.cache/elevation/SRTM1'
make: Nothing to be done for 'download'.
make: Leaving directory '/home/ahmad/.cache/elevation/SRTM1'
make: Entering directory '/home/ahmad/.cache/elevation/SRTM1'
make: Nothing to be done for 'all'.
make: Leaving directory '/home/ahmad/.cache/elevation/SRTM1'
make: Entering directory '/home/ahmad/.cache/elevation/SRTM1'
cp SRTM1.vrt SRTM1.3366522d2f104a049b2157fc879ac0fd.vrt
make: Leaving directory '/home/ahmad/.cache/elevation/SRTM1'
make: Entering directory '/home/ahmad/.cache/elevation/SRTM1'
gdal_translate -q -co TILED=YES -co COMPRESS=DEFLATE -co ZLEVEL=9 -co PREDICTOR=2 -projwin 10.40933 51.9003 10.42933 51.880300000000005 SRTM1.3366522d2f104a049b2157fc879ac0fd.vrt /home/ahmad/personal_ws/src/ROSExamples/script/ROI-DEM.tif
rm -f SRTM1.3366522d2f104a049b2157fc879ac0fd.vrt
make: Leaving directory '/home/ahmad/.cache/elevation/SRTM1'
<class 'numpy.ndarray'>
int32
(72, 72)
[[339 338 337 ... 292 293 294]
 [341 340 338 ... 

ERROR 1: PROJ: proj_create_from_name: Open of /home/ahmad/anaconda3/envs/debug/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /home/ahmad/anaconda3/envs/debug/share/proj failed


In [26]:
import rasterio as rio
from rasterio.warp import Resampling
upscale_factor = 1
dem_data = rio.open(fname)
dem_array = dem_data.read(1,
                            out_shape=(int(dem_data.height * upscale_factor),
                                        int(dem_data.width * upscale_factor)),
                            out_dtype=np.double,
                            resampling=Resampling.bilinear
                            )

# DEM image transform
print(dem_data.transform)
dem_transform = dem_data.transform * dem_data.transform.scale(
    (dem_data.width / dem_array.shape[-1]),
    (dem_data.height / dem_array.shape[-2])
)
print(bbx)
print(dem_transform[0])


| 0.00, 0.00, 10.41|
| 0.00,-0.00, 51.90|
| 0.00, 0.00, 1.00|
(10.40933, 51.880300000000005, 10.42933, 51.9003)
0.0002777777777777778


In [24]:
import numpy as np
# input data
ins = [[1, 1], [2, 3], [3, 2]]  # <- points
out = [[0, 2], [1, 2], [-2, -1]]  # <- mapped to
# calculations
l = len(ins)
B = np.vstack([np.transpose(ins), np.ones(l)])
D = 1.0 / np.linalg.det(B)


def entry(r, d): return np.linalg.det(
    np.delete(np.vstack([r, B]), (d+1), axis=0))


M = [[(-1)**i * D * entry(R, i) for i in range(l)] for R in np.transpose(out)]
A, t = np.hsplit(np.array(M), [l-1])
t = np.transpose(t)[0]
# output
print("Affine transformation matrix:\n", A)
print("Affine transformation translation vector:\n", t)
# unittests
print("TESTING:")
for p, P in zip(np.array(ins), np.array(out)):
    image_p = np.dot(A, p) + t
    result = "[OK]" if np.allclose(image_p, P) else "[ERROR]"
    print(p, " mapped to: ", image_p, " ; expected: ", P, result)


Affine transformation matrix:
 [[-1.66666667  1.33333333]
 [-2.          1.        ]]
Affine transformation translation vector:
 [0.33333333 3.        ]
TESTING:
[1 1]  mapped to:  [-2.22044605e-16  2.00000000e+00]  ; expected:  [0 2] [OK]
[2 3]  mapped to:  [1. 2.]  ; expected:  [1 2] [OK]
[3 2]  mapped to:  [-2. -1.]  ; expected:  [-2 -1] [OK]
